In [ ]:
!pip install pynvml GPUtil

In [ ]:
import os
import cv2
import time
import numpy as np
import psutil
from skimage.metrics import peak_signal_noise_ratio as psnr
import GPUtil

def non_local_means_denoising(image, h=10, hForColorComponents=10, templateWindowSize=7, searchWindowSize=21):
    start_time = time.time()
    if len(image.shape) == 2:
        denoised_image = cv2.fastNlMeansDenoising(image, None, h, templateWindowSize, searchWindowSize)
    else:
        denoised_image = cv2.fastNlMeansDenoisingColored(image, None, h, hForColorComponents, templateWindowSize, searchWindowSize)
    inference_time = (time.time() - start_time) * 1000
    return denoised_image, inference_time

def calculate_psnr(original_image, denoised_image):
    return psnr(original_image, denoised_image)

def process_images(input_folder, output_folder):
    total_inference_time = 0
    total_psnr = 0
    image_count = 0

    for filename in os.listdir(input_folder):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)

            denoised_image, inference_time = non_local_means_denoising(image)
            psnr_value = calculate_psnr(image, denoised_image)

            output_image_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_image_path, denoised_image)

            total_inference_time += inference_time
            total_psnr += psnr_value
            image_count += 1

    avg_inference_time = total_inference_time / image_count if image_count else 0
    avg_psnr = total_psnr / image_count if image_count else 0

    return avg_inference_time, avg_psnr, image_count

def print_gpu_metrics():
    gpus = GPUtil.getGPUs()
    for gpu in gpus:
        gpu_memory_allocated = gpu.memoryUsed
        gpu_memory_reserved = gpu.memoryTotal
        gpu_load = gpu.load * 100
        print(f"GPU Name: {gpu.name}")
        print(f"GPU Load (%): {gpu_load:.2f}%")
        print(f"GPU Memory Usage (MB): {gpu_memory_allocated:.2f} MB / {gpu_memory_reserved:.2f} MB")

def main():
    root_folder = '/content/drive/MyDrive/dataset'
    output_folders = ['denoised5', 'denoised15', 'denoised35']
    input_folders = ['noisy5', 'noisy15', 'noisy35']

    total_images = 0
    total_avg_inference_time = 0
    total_avg_psnr = 0

    for input_folder, output_folder in zip(input_folders, output_folders):
        input_path = os.path.join(root_folder, input_folder)
        output_path = os.path.join(root_folder, output_folder)
        os.makedirs(output_path, exist_ok=True)

        avg_inference_time, avg_psnr, image_count = process_images(input_path, output_path)

        total_avg_inference_time += avg_inference_time * image_count
        total_avg_psnr += avg_psnr * image_count
        total_images += image_count

    if total_images:
        total_avg_inference_time /= total_images
        total_avg_psnr /= total_images

    print(f"Total Images Processed: {total_images}")
    print(f"Average Inference Time per Image: {total_avg_inference_time:.2f} ms")
    print(f"Average PSNR (dB): {total_avg_psnr:.2f} dB")

    print_gpu_metrics()

if __name__ == "__main__":
    main()